
<h1>Urban Network Analysus (UNA) Tools</h1>

<h2> Example 1: Setting up a Network and Measuring Pedestrian Accissibility</h2>
<h3> Learning Objectives </h3>
In this example, we will learn how to
<il>
    <li> Get comfortable with the setup </li>
    <li> Loading data, handling layers and visualizing output </li>
    <li> Creating network from a layer, inserting origins and destinations, creating a network. </li>
    <li> Run basic accissibility measures: Reach and Gravity </li>
    <li> Use the service area and the closest destination UNA tools </li>

<h3> Getting comfortable with the setup </h3>
In order to use the UNA tools in Python, we use a spatial analysis library called Madina (Arabic for city). a Madina object holds data in a manner similar to the layering system used in GIS and Rhino. to use Madina, we first create an object and start loading data layers onto it.

In [ ]:
%config Completer.use_jedi = False 
import sys
sys.path.append('../')
from madina import Zonal
import madina.una.tools as una

# top-level imports
#from madina.zonal import Zonal
#import madina.una as una

cambridge = Zonal()

Since there is no GUI ()graphical user interface), whenever you need to know details about your Zonal object, call the <code>describe()</code> function

In [ ]:
cambridge.describe()

<h2>Loading Data Layers</h2>
Just like GIS and Rhino, the way to incorporate data into your analysis is through loading data in a layering system. The library supports different formats like <code>.geoJSON</code> and <code>.shp</code>. To load a layer, call the function <code>load_layer()</code> giving it a layer name for your new layer, and a path to where the data is stored.

In [ ]:
cambridge.load_layer(
    layer_name="building_entrances",
    file_path="./Data/building_entrances.geojson"
)
cambridge.load_layer(
    layer_name="subway",
    file_path="./Data/subway.geojson"
)
cambridge.load_layer(
    layer_name="sidewalks",
    file_path="./Data/sidewalks.geojson"
)

<h2>Looking inside</h2>
There are two ways to look at your data:
<ul>
<li><b>Visual Map:</b> creating a map is easy. Just call the <code>.create_deck_map()</code>function. to save the map as a file, call the same function, with a <code>save_as=cambridge_map.html"</code>. Colors are set randomly, but we'll go over map styling in a later lab. The current map visualizations utilize a library called <a href="https://deckgl.readthedocs.io">PyDeck</a>, a python wrapper for the javascript library <a href="https://deck.gl/">Deck.GL</a> open-sourced by a <a href="https://www.uber.com/blog/deck-gl-4-0/">major ride-hailing provider</a> that's based on the <a href="https://en.wikipedia.org/wiki/WebGL">WebGL<a> powertful 3D visualization framework made by a <a href="https://www.khronos.org/">non-profit organization</a>. Take this moment to appreciate the value of open source work here that made this possible (and free) and consider pushing for more open sourced  projects in your future life. </li>
<li><b>Text:</b> This would come handy when you need to know some details about your layers. Call the function <code>cambridge.describe()</code> to get a summary of your layers, their current settings and what columns are in each layer. This would be very handy when you're trying to debug for errors, especially when you start using data from external sources that might have different naming conventions, or when you're trying to deal with <a href="https://en.wikipedia.org/wiki/Spatial_reference_system">coordinate reference systems</a> or <a href="https://en.wikipedia.org/wiki/Map_projection">projection systems</a>.</li>
</ul>

In [ ]:
cambridge.create_map()

You can easily specify a color for each layer:

In [ ]:
cambridge.create_map(
    [
        {'layer': 'sidewalks', 'color': [125, 125, 125]}, 
        {'layer': 'subway', 'color': [255, 0, 0]}, 
        {'layer': 'building_entrances', 'color': [0, 0, 255]}
    ]
)

In [ ]:
cambridge.describe()

<h2> Creating a network </h2>
Networks (i.e. graphs) are a groups of nodes (i.e. vertices) and edges. The sidewalk network we loaded earlier might look graphically like a network, but for it to be useful, we need to convert it to a topologically connected sets of nodes and edges. Calling the <code>create_network_nodes_edges()</code> function does exactly that, we just need to give it a parameter <code>source_layer="sidewalks"</code> to let it know we're constructing a network from the <code>sidewalk layer</code>. When called, this function create two layers internally: <code>network_nodes</code> and <code>network_edges</code>. the network_edges table is equivelant to what's known as <a href="https://en.wikipedia.org/wiki/Edge_list">an edge list</a> in graph theory.

In [ ]:
cambridge.create_street_network(source_layer="sidewalks")

to see what the network looks like, we can call the <code>cambridge.create_map()</code> giving it a list of the layers we want to see.

In [ ]:
cambridge.create_map(
    layer_list=[
        {'gdf': cambridge.network.nodes, 'color': [255, 0, 255]}, 
        {'gdf': cambridge.network.edges, 'color': [125, 125, 125]}
    ]
)

<h2>Inserting Origins and Destinations</h2>
In urban networks, we will use the terms origins and destinations frequently. Origins are where <a href="https://en.wikipedia.org/wiki/Trip_generation">trips are generated</a>, and destinations are where trips are <a href="https://en.wikipedia.org/wiki/Trip_distribution">distributed to</a>. These are the first two steps of the <a href="https://en.wikipedia.org/wiki/Transportation_forecasting#Four-step_models">Four-step model</a> for traffic simulation. The third step is <a href="https://en.wikipedia.org/wiki/Mode_choice">mode choice</a>, which for the purposes of this library, is assumed to all be pedestrian trips. The fourth part is <a href="https://en.wikipedia.org/wiki/Route_assignment">route assignment</a> which we will learn more about when we talk about a generalization of the <a href="https://en.wikipedia.org/wiki/Betweenness_centrality"> betweenness algorithm </a> for route assignment.

In [ ]:
# inserting origins:
cambridge.insert_node(
    label='origin',
    layer_name="subway",
)

# inserting destinations
cambridge.insert_node(
    label='destination',
    layer_name="building_entrances",
)

As a convention in this library, there are a few default colors when visualizing the <code>network_nodes</code> and <code>network_edges</code> layers:
<ul>
<li>Origins are blue</li>
<li>Destinations are red</li>
<li>Network edges (road segments, sidewalk segments,...) are shown in a dark gray for intersection nodes, and lighter gray for network segments</li>
<li>If you don't see an intersection node when you expect people being able to from an edge to another one, the network is not topologically correct, and some possible routes would be missed. in the Rhino part A of this excercise, we learned how to clean up a network. Whem working with this library, we assume the input to <code>create_network_nodes_edges()</code> is a clean geometry where the end of a line exactly touches the begenning of another one to establish a connection. </code></li>
</ul

In [ ]:
cambridge.create_map(
    layer_list=[
        {'gdf': cambridge.network.nodes, 'color_by_attribute': 'type', 'color_method': 'categorical'}, 
        {'gdf': cambridge.network.edges, 'color': [125, 125, 125]}
    ]
)

<h2>Constructing a graph </h2>
The last step before being able to do an analysis, is to commit to a finalized graph. once you're done creating your network, and inserted your origins and destinations, you can call the <code>create_graph()</code> function that creates a "graph" object for your analysis.

In [ ]:
cambridge.create_graph()

<h2>Analyze how many building entrances or people can be reached from subway
stations in a 5-minute (300m) walkshed.</h2>
From now on, we will attempt doing the same analysis done on the Rhino part B of this exercise. THe following steps are identical to the Rhino part, and the same intuition and mathematical representation still holds.

<h2>Reach Index</h2>
Running the <code>una_accessibility</code> and giving it parameters <code>reach=True, search_radius=300</code> will measure how many how many destinations (building entrances) are reachable from origins (subway).
<p>

We see that the northen station could be reached by 106 building entrance, while the southern station could be reached by 112 building entrance.
<p>

The <code>una_accessibility</code> creates a column called <code>una_reach</code> when the parameter <code>reach=True</code>. We could use that to explore more visualization functionalities:

In [ ]:
una.accessibility(
    cambridge,
    reach=True,
    search_radius=300
)


cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes, "radius": "una_reach", 'text':'una_reach', 'color': [255, 0, 0]},
    ]
)


Setting the parameter <code>weight='people'</code> enables us to weight the destinations (building entrances) by how many people actually live in these building. We could then see that 2,789 people could reach the northen station by walking a maximum of 300 meters. The southern station could be reached by 3,017 people.

In [ ]:
una.accessibility(
    cambridge,
    reach=True,
    search_radius=300,
    weight='people'
)

cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes, "radius": "una_reach", 'text':'una_reach', 'color': [255, 0, 0]},
    ]
)

<h2> Service Area </h2>
Sometimes, we want to examine a section of our data in isolation. The <code>service_area()</code> function is a great introduction into that. The library internally uses a library called <a href="https://geopandas.org/en/stable/">GeoPandas</a>, where a layer is usually stored as a GeoDataFrame. GeoPandas are an extenstion to the well known <a href="https://pandas.pydata.org/">Pandas library</a>. If you're familiar with the Pandas Dataframe, you'll be able to use all what you know about a Dataframe. The only difference is that a GeoDataFrame always has a <code>geometry</code> columns, and supports a wide array of spatial operations.
<p>

The <code>service_area()</code> function returns a GeoDataFrame <code>destinations</code> containing all the destinations covered by an origin's service area, and a GeoDataframe <code>network_edges</code> containing all network segments inside the origin's service area. A Third Dataframe <code>scope_gdf</code> contains the boundaries of the service area. Notice how the function <code>create_deck_map()</code> could take either a <code>layer</code> from the layers we loaded, or a <code>gdf</code> (i.e. GeoDataFrame) resulting from a process.

In [ ]:
destinations, network_edges, scope_gdf = una.service_area(
    cambridge,
    #[120],
    search_radius=100,
)

cambridge.create_map(
    layer_list=[
        {"layer": 'sidewalks'},
        {"layer": 'building_entrances'},
        {"gdf": network_edges, "color": [0, 255, 0]},
        {"gdf": destinations, "color": [255, 0, 0]},
        {"gdf": scope_gdf[scope_gdf['name'] == 'service area border'], "color": [0, 0, 255], 'opacity': 0.10},
    ]
)


<h2> Setting up Attributes </h2>
Generally, in a coding environment, you would rarely need to change or set one single attribute of one single element in a layer. Attributes are usually either pure data inputs, or a calculated result that is not set manually. In this contexct, setting up one single attribute might be really valuble when you are debugging your work, want to validate the math/logic or just want to experament with specific values. The <code>set_attribute()</code>  function allows for that.
<p>

While in a visual interface, it is possible to select the elements you want with a mouse. In a coding environment, every object have an identifier, and each layer once loaded is assigned an <code>id</code> attribute. Hover over tha previous map to find the building entrances with ids 2 and 115, They are the same one referref to in the Rhino bullet point 19. They are both in <code>layer="building_entrances</code> and we want to set a <code>attribute='student'</code> to values 10 and 10, for ids 2 and 115 respectively.
<p>

From the map below, we now notice that 20 students can reach the northen station, and 10 students could reach he southern station on a 300 meter walk.

In [ ]:
cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes.reset_index(), 'text':'id', 'color': [255, 0, 0]},
    ]
)

In [ ]:
cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes.loc[[2, 115]].reset_index(), 'text':'id', 'color': [255, 0, 0]},
    ]
)

In [ ]:
cambridge.layers['building_entrances'].gdf.at[2, 'students'] = 10
cambridge.layers['building_entrances'].gdf.at[115, 'students'] = 10



una.accessibility(
    cambridge, 
    reach=True,
    search_radius=300,
    weight='students'
)
cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes, "radius": "una_reach", 'text':'una_reach', 'color': [255, 0, 0]},
    ]
)

<h2> Gravity Index </h2>
The <a href="https://en.wikipedia.org/wiki/Trip_distribution#Gravity_model"> gravity model</a> of transportation is one of many <a href="https://en.wikipedia.org/wiki/Distance_decay">distance decay</a> functions. The aim of a distance decay function (e.g. gravity) is to quantify the inverse relationship between distance, and willingness to make trips. applying gravity peanilize further destinations by reducing their contribution, assume that they are less accissable. For this reason, a gravity index is always less than or equal to a reach index.
<p>

When factoring in a gravity decay, we notice that the northen station only attracts 16.88 students (As opposed to 20) and the southern station now attracts only 8.67 students (as opposed to 10).

In [ ]:
una.accessibility(
    cambridge, 
    gravity=True,
    search_radius=300,
    beta=0.001,
    alpha=1,
    weight='students',
)

cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes, "radius": "una_gravity", 'text':'una_gravity', 'color': [255, 0, 0]},
    ]
)

<h2>Comparing Reach and Gravity of People to Subway</h2>
We go back to using <code>people</code> as weight, and we again call the function <code>una_accessibility()</code>, this time, setting both reach and gravity to <code>True</code>
<p>

Hovering over the map below to where the stations are, we could see that the northern station has a reach index of  2,789 bur a gravity index of only 2,361.54, The southern station has a reach index of 3,017 and a gravity_index of only 2,606.81.


In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    gravity=True,
    search_radius=300,
    beta=0.001,
    alpha=1,
    weight='people',
)
cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes, "radius": "una_gravity", 'text':'una_gravity', 'color': [255, 0, 0]},
    ]
)

<h2> Closest Facility </h2>
<p>
One note about the <code>una_accessibility()</code>, there will be overlap between destinations assigned to each origin, so a destination would be double counted in an origin's accissability indices. The <code>closest_facility()</code> assigns each  destination (in this case, building entrance) to one single origin (a subway station in this case)
</p>
<p>
<table border="1" cellpadding="2">
<tbody>
<tr>
<td>Function</td>
<td colspan="2"><code>una.accessibility()</code></td>
<td colspan="2"><code>una.closest_facility()</code></td>
</tr>
<tr>
<td>Station</td>
<td>Reach</td>
<td>Graviy</td>
<td>Reach</td>
<td>Gravity</td>
</tr>
<tr>
<td>Northren</td>
<td>2,789.00</td>
<td>2.361.54</td>
<td>1,265.00</td>
<td>1,120.92</td>
</tr>
<tr>
<td>Southren</td>
<td>3,017.00</td>
<td>2,606.81</td>
<td>1,914.00</td>
<td>1,716.57</td>
</tr>
</tbody>
</table>

</p>
<p>
This variation of values (across the same station), poses the question of What value is closest to reality? This is an ongoing research topic, and urban scientists are able to choose better models the more we learn about pedestrian activity through data,
</p>

In [ ]:
una.closest_facility(
    cambridge,
    weight='people',
    gravity=True,
    reach=True,
    search_radius=300,
    beta=0.001
)

cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]},
        {"gdf": cambridge.network.nodes,"text": "una_closest_destination_distance",  "color_by_attribute": "una_closest_destination", "color_method": "categorical"},#, 'radius': "una_reach"},
    ]
)

<h2>What About the Reverse? Reach and Gravity of Subway to People </h2>
To examine the different interpretation of accessibility indices based on what we define as origins and destinations, lets flip our origins from <code>subway</code> to <code>building_entrances</code> and destinations from <code>building_entrances</code> to <code>subway</code>.
<p>
In order to do this, we need to first clear up the origins and destination nodes we created earlier, bur we could still use the same neteowk edges (sidewalks) as before, since changing the origins and destinations will not impact the underlying network segments and intersections. Calling the function <code>clear_nodes()</code> empties out our existing oeigins and destinations. We then need to insert origins and destinations, then create a graph object, just like before.

In [ ]:
cambridge.clear_nodes()
cambridge.insert_node(
    label='origin',
    layer_name="building_entrances",
)
cambridge.insert_node(
    label='destination',
    layer_name="subway",
)
cambridge.create_graph()

The reach index shown below, shows how many subway station could be reached from each building enterance, by walking 300 meters. Most building entrances can access two stations, but buildings on the phrepherey, could only access one of the two stations within a 200 meter walk,

In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    search_radius=300,
)

cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.nodes, "color_by_attribute": "una_reach", 'color_method': 'gradient',  'text':'una_reach'},
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]}
    ]
)

In [ ]:
cambridge.layers['subway'].gdf.at[0, 'daily_trains'] = 20
cambridge.layers['subway'].gdf.at[1, 'daily_trains'] = 50


una.accessibility(
    cambridge, 
    reach=True,
    search_radius=300,
    weight='daily_trains'
)

cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.nodes, "color_by_attribute": "una_reach", 'color_method': 'gradient',  'text':'una_reach'},
        {"gdf": cambridge.network.edges, 'color': [125, 125, 125]}
    ]
)

As for the gravity index, the interpretation of what these values mean is sensitive to the bets parameter which could translate to the "area's willingness to walk". living in a building with a high gravity index, means that you have more subway options that are close and could easily be reached on foot. A low gravity index, means that you live in a building with fewer subway options that are further out.

In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    gravity=True,
    search_radius=300,
    beta=0.004,
)
cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.nodes, "color_by_attribute": "una_gravity", 'color_method':'gradient', 'text': 'una_gravity'},
        {"gdf": cambridge.network.edges, 'color': [135, 125, 125]}
    ]
)

In [ ]:
una.accessibility(
    cambridge, 
    reach=True,
    gravity=True,
    search_radius=300,
    beta=0.004,
    weight='daily_trains'
)
cambridge.create_map(
    layer_list=[
        {"gdf": cambridge.network.nodes, "color_by_attribute": "una_gravity", 'color_method':'gradient', 'text': 'una_gravity'},
        {"gdf": cambridge.network.edges, 'color': [135, 125, 125]}
    ]
)
